## Runnable Primitives

### Runnable Sequence (|)


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableSequence
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()


model= ChatGroq(api_key=groq_api_key, model="openai/gpt-oss-120b")

prompt1= PromptTemplate(
    template='Generate a LinkedIn post about {topic}',
    input_variables=['topic']
)

prompt2= PromptTemplate(
    template='Generate a Twitter post about {topic}',
    input_variables=['topic']
)

parser= StrOutputParser()

parallel_chain= RunnableParallel({
    'tweet': RunnableSequence(prompt1,model,parser),
    'linkedin_post': RunnableSequence(prompt2,model,parser)
})

response= parallel_chain.invoke({'topic': 'AI in Healthcare'})
print(response)

d:\Projects\Python\genai\langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'tweet': '🚀 **AI is Transforming Healthcare – and the Future Is Already Here** 🚀  \n\nFrom early disease detection to personalized treatment plans, artificial intelligence is reshaping every corner of the health ecosystem. Here are three game‑changing trends I’m seeing right now:\n\n1️⃣ **Predictive Analytics & Early Diagnosis** – Machine‑learning models can spot subtle patterns in imaging, lab results, and EHR data that even seasoned clinicians might miss. Think of AI as a “second pair of eyes” that flags potential issues weeks or months before symptoms appear.\n\n2️⃣ **Personalized Medicine at Scale** – By integrating genomics, lifestyle data, and real‑time monitoring, AI helps craft treatment regimens tailored to each patient’s unique biology. This not only boosts efficacy but also reduces adverse events.\n\n3️⃣ **Operational Efficiency & Burnout Reduction** – Automation of routine tasks—appointment scheduling, billing, documentation—frees clinicians to focus on what truly matters:

### RunnablePassThrough

Simply returns the input as the output. Done so as to preserve the structure not as a logical operation.
When used with the assign() operation, it modifies the given property of a dict and keeps the rest untouched.

In [2]:
from langchain_core.runnables import RunnablePassthrough

passthrough= RunnablePassthrough()
result= passthrough.invoke("This is a test.")
print(result) 
print(type(result)) 

prompt1= PromptTemplate(
    template='Generate a joke about this {topic}',
    input_variables=['topic']
)


prompt2= PromptTemplate(
    template='Explain this {joke}',
    input_variables=['joke']
)

joke_generator_chain= RunnableSequence(prompt1,model,parser)

parallel_chain= RunnableParallel({
    "joke": RunnablePassthrough(),
    "explanation":RunnableSequence(prompt2,model,parser)
})

result=RunnableSequence(joke_generator_chain,parallel_chain)
result.invoke({'topic':'Footballers'})

This is a test.
<class 'str'>


{'joke': 'Why did the footballer bring a ladder to the match?\n\nBecause he heard the stakes were high and wanted to make sure he could **reach** the top of the league! 😄',
 'explanation': '**How the joke works**\n\n1. **Setup (the question)**  \n   *“Why did the footballer bring a ladder to the match?”*  \n   This creates a mental picture of a player showing up with an odd piece of equipment. The listener expects a funny, unexpected reason.\n\n2. **Punchline (the answer)**  \n   *“Because he heard the stakes were high and wanted to make sure he could **reach** the top of the league!”*  \n\n3. **Why it’s funny – the wordplay**\n\n   | Word / Phrase | Literal meaning | Figurative meaning in football |\n   |---------------|----------------|--------------------------------|\n   | **stakes**   | The wooden posts that hold a fence or a “stake” you drive into the ground. In everyday speech it also means “the level of risk or reward.” | In a competition, “high stakes” means there’s a lot on t

### Runnable Lambda

Converts any python function into a runnable primitive  that allows us to use it in the AI pipeline

In [3]:
from langchain_core.runnables import RunnableLambda

def count_words(words:str)->int:
    count=0
    words=words.strip()
    print(words.split()) # could have used this also, stupid me
    for word in words:
        if word == ' ':
            count+=1

    return count+1


joke_generator_chain= RunnableSequence(prompt1,model,parser)

parallel_chain=(
    {
        "joke":RunnablePassthrough(),
        "words": RunnableLambda(count_words)
    }
)

final_chain=RunnableSequence(joke_generator_chain,parallel_chain)

result=final_chain.invoke({"topic":"Computers"})
result

['Why', 'did', 'the', 'computer', 'go', 'to', 'therapy?', 'Because', 'it', 'had', 'too', 'many', '*bytes*', 'of', 'emotional', 'baggage', 'and', 'kept', 'crashing', 'under', 'the', 'pressure', 'of', 'its', 'own', '*cache*', 'of', 'feelings!']


{'joke': 'Why did the computer go to therapy?\n\nBecause it had too many *bytes* of emotional baggage and kept crashing under the pressure of its own *cache* of feelings!',
 'words': 27}

### Runnable Branch

It offers a conditional flow component to the Langchain that allows us to conditionally route input data to different chains or runnables based on custom logic

In [5]:
from langchain_core.runnables import RunnableBranch

prompt1= PromptTemplate(
    template="Write a detailed report on {topic}",
    input_variables=["topic"]
)

prompt2= PromptTemplate(
    template="Summarize the following text \n {text}",
    input_variables=['text']
)

report_generator_chain= RunnableSequence(prompt1,model,parser)

branch_chain= RunnableBranch(
    (lambda x: len(x.split())>200, RunnableSequence(prompt2,model,parser)),
    RunnablePassthrough()
)

final_chain= RunnableSequence(report_generator_chain,branch_chain)

response=final_chain.invoke({"topic":"English Football"})
response

'**English Football – Key Take‑aways**\n\n| Topic | Main Points |\n|-------|--------------|\n| **1. Overview** | Football is a cultural pillar in England, evolving from 19th‑century public‑school games into a multi‑billion‑pound global industry. |\n| **2. Historical Milestones** | • 1848 – Cambridge Rules (first written rules). <br>• 1863 – FA founded, Laws of the Game standardized. <br>• 1888 – Football League created (12 clubs). <br>• 1992 – Premier League launched, ushering massive TV deals. <br>• 1971 onward – Women’s football revived; 2022 England win Euro. |\n| **3. Governance** | • **FA** (est. 1863) – regulator, national teams, grassroots. <br>• **League System** – 5‑tier professional pyramid (Premier League → National League) with promotion/relegation. <br>• **Cup competitions** – FA Cup (all levels), EFL Cup, Community Shield, EFL Trophy, Women’s Cup. |\n| **4. Economic Landscape** | • Premier League revenue split: ~55\u202f% broadcasting, 20\u202f% matchday, 25\u202f% commer